Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [519]:
from random import choices
from dataclasses import dataclass
from copy import copy
from random import randint, random, choice
import lab9_lib

In [520]:
fitness = lab9_lib.make_problem(10)

for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")


print(fitness.calls)

11001010010110100101001100001001101111000111100100: 23.36%
01101001001000000001110011010001011101011101110110: 23.34%
10011011101101000001001111110111101010000010011001: 7.33%
11010111011100000100001110001010001001000100110100: 7.33%
10110101001011001100101011001110010010010010000010: 15.36%
01101110110101110101111110000010110010011100110010: 15.33%
01110001110101011111100001010011110000010011101011: 15.33%
01010000010010001000001100100100100100111110010011: 15.56%
11010010001110001000010101010000110010101101011100: 23.36%
10000100000110011110111111000011010011110001000000: 23.56%
10


In [521]:
POPULATION_SIZE = 30
OFFSPRING_SIZE = 20
TOURNAMENT_SIZE = 10
MUTATION_PROBABILITY = .15

SEQUENCE_LENGTH = 1000

In [522]:
@dataclass
class Individual:
    fitness: float
    genotype: list[bool]

### Selection

In [523]:
class Tournament():
    def __init__(self, size):
        self.size = size

    def select(self, pop):
        pool = [choice(pop) for _ in range(self.size)]
        champion = max(pool, key=lambda i: i.fitness)
        return champion
    
class TournamentEpsilon():
    def __init__(self, size, scheduler):
        self.size = size
        self.schedular = scheduler
    
    def select(self, pop):
        if random() > self.schedular.value():
            pool = [choice(pop) for _ in range(self.size)]
            champion = max(pool, key=lambda i: i.fitness)
            return champion
        else:
            return choice(pop)

### Mutation

In [524]:
class Scheduler():
    def __init__(self, alpha, decay_rate = 0):
        self.rate = decay_rate
        self.current_value = alpha
    
    def mutation_prob():
        pass

class ConstantScheduler(Scheduler):
    def value(self):
        return self.current_value

class ExponentialDecayScheduler(Scheduler):
    def value(self):
        return_value = self.current_value
        self.current_value = self.current_value * (1 - self.rate)
       
        return return_value
    
class BitFlip():
    def __init__(self, sequence_length, scheduler):
        self.sequence_length = sequence_length
        self.scheduler = scheduler
    
    def mutate(self, ind: Individual) -> Individual:
        offspring = copy(ind)

        if random() < self.scheduler.value():
            print('mutate happen')
            pos = randint(0, self.sequence_length-1)
            offspring.genotype[pos] = not offspring.genotype[pos]
            offspring.fitness = None
        
        return offspring

### Recombination

In [525]:
class OnePointCrossover():
    def __init__(self, sequence_length):
        self.sequence_length = sequence_length

    def recombine(self,ind1: Individual, ind2: Individual) -> Individual:
        cut_point = randint(0, self.sequence_length-1)
        offspring1 = Individual(fitness=None,
                            genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:])
        offspring2 = Individual(fitness=None,
                            genotype=ind2.genotype[:cut_point] + ind1.genotype[cut_point:])

        assert len(offspring1.genotype) == self.sequence_length
        assert len(offspring1.genotype) == self.sequence_length

        return offspring1, offspring2

class UniformCrossover():
    def __init__(self, sequence_length):
        self.sequence_length = sequence_length
    
    def recombine(self, ind1: Individual, ind2: Individual) -> Individual:
        rvector1 = [random() for _ in range(self.sequence_length)]
        rvector2 = [random() for _ in range(self.sequence_length)]

        g1 = [p1 if r < 0.5 else p2 for p1, p2, r in zip(ind1.genotype, ind2.genotype, rvector1)]
        g2 = [p1 if r < 0.5 else p2 for p1, p2, r in zip(ind1.genotype, ind2.genotype, rvector2)]

        o1 = Individual(fitness = None, genotype=g1)
        o2 = Individual(fitness = None, genotype=g2)

        return o1, o2



In [526]:
class EA_Simulation():
    def __init__(self, population_size, offspring_size, fitness_eval,selection, mutation, recombination, num_gen = 100):
        self.population_size = population_size
        self.offspring_size = offspring_size
        self.fitness_eval = fitness_eval
        self.select_parent = lambda pop : selection.select(pop)
        self.mutate = lambda ind : mutation.mutate(ind)
        self.recombine = lambda ind1, ind2: recombination.recombine(ind1, ind2)
        self.num_gen = num_gen

    def set_inital_pop(self, population):
        self.population = population

    def simulate(self):
        record = []

        for _ in range(self.num_gen):
            offspring = list()

            #Evolution process
            for _ in range(self.population_size // 2):
                p1 = self.select_parent(self.population)
                p2 = self.select_parent(self.population)

                o1, o2= self.recombine(p1, p2)

                offspring.append(self.mutate(o1))
                offspring.append(self.mutate(o2))

            #Evaluate offsprings
            for i in offspring:
                i.fitness = fitness(i.genotype)

            #Add offsprings into the population and choose the best ones for the next generation  
            self.population.extend(offspring)
            self.population.sort(key=lambda i: i.fitness, reverse=True)
            self.population = self.population[:self.population_size]

            record.append(self.population[0].fitness)

        return record
        

        
        

### Initialize Population

In [527]:
population = [
    Individual(
        genotype=[choice((True, False)) for _ in range(SEQUENCE_LENGTH)],
        fitness=None,
    )
    for _ in range(POPULATION_SIZE)
]

for i in population:
    i.fitness = fitness(i.genotype)

### Experiment

In [528]:
selection_scheduler = ExponentialDecayScheduler(alpha = 1, decay_rate = 0.000001)
selection = TournamentEpsilon(size = SEQUENCE_LENGTH, scheduler=selection_scheduler)

mutation_scheduler = ExponentialDecayScheduler(alpha=1, decay_rate=0.000001)
mutation = BitFlip(sequence_length= SEQUENCE_LENGTH, scheduler=mutation_scheduler)
recombination = UniformCrossover(sequence_length= SEQUENCE_LENGTH)

sim = EA_Simulation(population_size=POPULATION_SIZE, offspring_size= OFFSPRING_SIZE, fitness_eval=fitness, selection=selection, mutation=mutation, recombination=recombination, num_gen=10000)

sim.set_inital_pop(population)

record = sim.simulate()

mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate

mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate happen
mutate

In [529]:
record

[0.001,
 0.003,
 0.005,
 0.005,
 0.005,
 0.0079,
 0.0079,
 0.0079,
 0.009890000000000001,
 0.009890000000000001,
 0.0099,
 0.0099,
 0.0099,
 0.0099,
 0.0099,
 0.0099,
 0.0099,
 0.011788999999999999,
 0.014889,
 0.014889,
 0.014889,
 0.014889,
 0.014889,
 0.014889,
 0.014889,
 0.014889,
 0.014889,
 0.014889,
 0.014889,
 0.01489,
 0.01489,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.018,
 0.018,
 0.018,
 0.018,
 0.018,
 0.018,
 0.018,
 0.018,
 0.018,
 0.018,
 0.018,
 0.018,
 0.018,
 0.0209,
 0.0209,
 0.0209,
 0.0209,
 0.0209,
 0.0209,
 0.0209,
 0.0209,
 0.0209,
 0.0209,
 0.0209,
 0.0209,
 0.0209,
 0.0209,
 0.0209,
 0.0238,
 0.0238,
 0.0238,
 0.0238,
 0.0238,
 0.0238,
 0.0238,
 0.0238,
 0.0238,
 0.0238,
 0.0238,
 0.0238,
 0.0238,
 0.0238,
 0.0238,
 0.0238,
 0.0238,
 0.0238,
 0.0238,
 0.0238,
 0.0238,
 0.0238,
 0.0238,
 0.0268,
 0.0268,
 0.0268,
 0.0268,
 0.0268,
 0.0268,
 0.0268,
 0.03,
 0.03,
 0.03,
 0.03,
 0.03,
 0.03,
 0.03,
 0.03,
 0.03,
 0.03,
 0.03,
 0.03,
 0.

In [530]:
scheduler.current_value

0.7408171094488248